https://arxiv.org/abs/1711.05772 / https://arxiv.org/abs/1802.04877

https://github.com/natashamjaques/magenta/blob/affective-reward/magenta/models/affective_reward/latent_gan.py

In [1]:
import torch

print('cuda.is_available:', torch.cuda.is_available())
print(f'available: {torch.cuda.device_count()}; current: {torch.cuda.current_device()}')
DEVICE = torch.device(f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
print('pytorch', torch.__version__)

cuda.is_available: True
available: 1; current: 0
cuda:0
pytorch 0.4.0


## Labeling data

In [2]:
# pip install git+https://github.com/iconix/pytorch-text-vae.git
from pytorchtextvae import generate

In [3]:
DEVICE = torch.device('cpu') # CPU inference
n_samples = 200
temp = 0.2

# workaround for un-pickling after module directory change https://stackoverflow.com/a/45264751
#import sys
#sys.path.append('../../pytorch-text-vae/pytorchtextvae')

vae, input_side, output_side, pairs, dataset, EMBED_SIZE, random_state = generate.load_model('../../pytorch-text-vae/model/best/reviews_and_metadata_5yrs_state.pt', 'reviews_and_metadata_5yrs_stored_info.pkl', DEVICE, cache_path='../../pytorch-text-vae/model/best/tmp')

Fetching cached info at ../../pytorch-text-vae/model/best/tmp/reviews_and_metadata_5yrs_stored_info.pkl
Cache ../../pytorch-text-vae/model/best/tmp/reviews_and_metadata_5yrs_stored_info.pkl loaded (load time: 0.59s)
Found saved model ../../pytorch-text-vae/model/best/reviews_and_metadata_5yrs_state.pt
MAX_SAMPLE: False; TRUNCATED_SAMPLE: True
Trained for 360000 steps (load time: 18.92s)
Setting new random seed


In [4]:
#generate.generate(vae, input_side, output_side, pairs, dataset, EMBED_SIZE, random_state, DEVICE, genres=['downtempo', 'dream pop', 'indietronica'], num_sample=10, temp=temp)

In [5]:
gens, zs, conditions = generate.generate(vae, input_side, output_side, pairs, dataset, EMBED_SIZE, random_state, DEVICE, num_sample=n_samples, temp=temp)

---
['chamber pop', 'chillwave', 'downtempo', 'electronic', 'freak folk', 'indie pop', 'indie r&b', 'indietronica', 'neo-psychedelic', 'wonky']
UNK is another perfect ability to UNK UNK r UNK artist is the perfect blend of soulful vocals and your instrumentation
---
['chillwave', 'indietronica', 'shimmer psych', 'vapor soul', 'vapor twitch']
i cant put through the fan of UNK when he returns of a good for me and her ability to feel good head
---
['electropop', 'indie electro-pop', 'indie poptimism', 'norwegian pop', 'vapor pop', 'vapor soul']
today with anna of the UNK which is set to get mood to some people in the anna video they your places
---
['electropop', 'indie electro-pop', 'indie poptimism', 'indietronica', 'vapor soul']
its like a a way that has made me even after all the things are set out on this different places
---
['electropop', 'indie poptimism', 'pop', 'swedish electropop', 'swedish pop', 'swedish synthpop']
artist are are in artist and progressive house and a house and

---
['dance-punk']
the UNK in themselves too in like or even synths gone with inspired synths
---
['electropop', 'indie poptimism', 'indie r&b', 'indietronica', 'vapor soul']
and it and produced the deep and before that deep in the the but after being sure to with some of the directed of least as well
---
['chamber pop', 'indie anthem-folk', 'indie folk', 'indie r&b', 'vapor soul']
there subtle enough of the fact that the subject matter how much of the UNK and the stripped back in the many in the familiar of him in the many of every work
---
['indie anthem-folk']
but artist has some releasing the original music in in UNK work and years sense
---
['alternative hip hop', 'escape room', 'hip hop', 'rap']
earlier there
---
['big room', 'edm', 'electro house', 'progressive house']
artist s signature inspired instrumentation and all of the combined with its strong taste of their human instrumentation
---
['australian pop', 'dance pop', 'electropop', 'hip pop', 'pop', 'post-teen pop', 'r&b', 

---
['pop rap']
UNK was released in UNK later this year you can also with some of the club thought you can can throughout
---
['alternative dance', 'aussietronica', 'deep house', 'filter house', 'house', 'nu disco', 'tropical house']
a brand of of last year artist is very coming in hailing from these artist
---
['indie anthem-folk']
it starts off over a bit of and and i fell in love which you over a soft soft in soft melody
---
['norwegian pop']
l a duo artist continues with the kind of artists such as UNK continues to such a killer track
---
['dance pop', 'edm', 'electropop', 'metropopolis', 'pop', 'post-teen pop', 'tropical house']
and title track is so kicked off in his own but before so many of his his vocals make a welcome addition to the light
---
['indie electro-pop', 'indie poptimism']
the the UNK song its going to love and its UNK thanks to its almost ahead of its tropical atmosphere
---
['chamber pop', 'folk-pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'modern

---
['funk', 'indie jazz', 'indie r&b', 'neo soul', 'soul']
and weve just been earlier this track for a few years ago and and over the soulful piece of la based track for their fun
---
['electropop', 'indie electro-pop', 'indie poptimism', 'indie psych-rock', 'indie r&b', 'vapor soul']
when the band off with her forthcoming debut band which is one in the month and it will get of your place
---
['art pop', 'chamber pop', 'indie pop', 'indie r&b', 'neo soul']
ever since i grew up in a long time and its much waiting for me
---
['indie psych-rock', 'metropopolis']
the track that the name stuck in your way and i will make like more like to you like like
---
['indie anthem-folk', 'indie poptimism', 'modern rock', 'pop']
i have a brand of UNK and UNK UNK is gearing up to UNK UNK UNK co produced by by ninja tune
---
['ninja', 'nu jazz']
UNK UNK UNK UNK is you to to its way to UNK UNK vocals at last ep tonight
---
['chamber pop', 'folk-pop', 'indie folk', 'indie pop', 'new americana', 'stomp an

---
['brooklyn indie', 'electropop', 'indie electro-pop', 'indie poptimism', 'indie r&b', 'indietronica', 'metropopolis', 'pop', 'shimmer pop', 'vapor pop', 'vapor soul']
looks like to putting out on september and and verite who has a la based original version
---
['metropopolis', 'shimmer pop', 'tropical house']
with the first sound and was inspired by kanye made in one in which is coming out later this remix
---
['alternative dance', 'australian dance', 'chillwave', 'dance-punk', 'electroclash', 'filter house', 'indie pop', 'indietronica', 'new rave', 'nu disco', 'shimmer pop']
the UNK is an about four days back but by the some of their soulful vocals just under vocals
---
['electropop', 'indie poptimism', 'shimmer pop']
the tune with lot of us of the subject matter of the dance tune we come into dance place


In [6]:
list(zip(range(len(gens)), gens))

[(0,
  'UNK is another perfect ability to UNK UNK r UNK artist is the perfect blend of soulful vocals and your instrumentation'),
 (1,
  'i cant put through the fan of UNK when he returns of a good for me and her ability to feel good head'),
 (2,
  'today with anna of the UNK which is set to get mood to some people in the anna video they your places'),
 (3,
  'its like a a way that has made me even after all the things are set out on this different places'),
 (4,
  'artist are are in artist and progressive house and a house and seem to break up there is one too'),
 (5,
  'the electro pop trio artist has made for the top of UNK artists in the debut ep'),
 (6, 'r UNK UNK the UNK trio has to me over UNK years of last artist'),
 (7,
  'its a song track taken from the bright guitars and soulful vocals shine over the instead of his atmosphere'),
 (8,
  'the newest track with these its because their sound and perfect addition to these trap sound'),
 (9,
  'one of the shaun frank ocean drive m

In [7]:
def to_embed(z, condition):
    if condition.dim() == 1:
        condition = condition.unsqueeze(0)
    squashed_condition = vae.decoder.c2h(condition)
    return torch.cat([z, squashed_condition], 1)

In [8]:
n_latent = 128
from pytorchtextvae.datasets import EOS_token

def generate(condition, gan=None, z=None, max_sample=False, truncated_sample=True, temp=temp):
    with torch.no_grad():
        if gan is None:
            z_prime = z
        else:
            gan.eval()
            z = torch.randn(1, n_latent).to(DEVICE)
            decode_embed = to_embed(z, condition).to(DEVICE)
            z_prime = gan.G(decode_embed)

        generated = vae.decoder.generate_with_embed(z_prime, 50, temp, DEVICE, max_sample=max_sample, trunc_sample=truncated_sample)
        generated_str = model.float_word_tensor_to_string(output_side, generated)

        EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

        if generated_str.endswith(EOS_str):
            generated_str = generated_str[:-5]

        # flip it back
        return generated_str[::-1], z, z_prime

### 'Banned' approach

label a sample as -1 (=="bad") if it contains a banned word; label as 1 otherwise (=="good")

In [7]:
#new_labels = np.array([(1, -1), (10, -1)])

banned = ['below']
labels = np.ones(n_samples, dtype=int)
gens_lose = list(set([i for b in banned for i in np.where([b in g.split() for g in gens])[0]]))
labels[gens_lose] = -1
zs_keep = zs

labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1])

In [8]:
from collections import Counter
c1 = Counter([word for gen in gens for word in gen.split()])
[(b, c1[b]) for b in banned], c1.most_common(25)

([('below', 3)],
 [('the', 94),
  ('UNK', 82),
  ('of', 67),
  ('a', 66),
  ('and', 59),
  ('to', 55),
  ('is', 29),
  ('as', 26),
  ('artist', 26),
  ('with', 26),
  ('from', 24),
  ('it', 21),
  ('this', 21),
  ('track', 20),
  ('on', 20),
  ('up', 19),
  ('one', 18),
  ('be', 15),
  ('in', 14),
  ('has', 14),
  ('at', 11),
  ('i', 10),
  ('been', 10),
  ('trying', 10),
  ('that', 10)])

### 'Realism' approach

label a sample as 1 (=="good") if it came from the training data; label as -1 (=="bad") if it came from a random Gaussian `z`

In [10]:
from pytorchtextvae import model

input, target, condition = model.random_training_set(pairs, input_side, output_side, random_state, DEVICE)
model.long_word_tensor_to_string(input_side, input), dataset.decode_genres(condition)

('newcomer artist released his debut single last week and its already gaining major attention and a following that is demanding more after fill EOS ',
 ['vapor soul'])

In [11]:
temperature = 1.0

m, l, z, decoded = vae(input, target, condition, DEVICE, temperature)

z.size(), decoded.size()

(torch.Size([1, 128]), torch.Size([24, 333336]))

In [12]:
generate(condition, z=z, max_sample=True)[0]

'artist released his debut single and is just released and more than a ago and that that get your attention'

**TODO:** shouldn't generate with max sampling always return the same sample?

Even though the encoding is imperfect, we will still consider these `z`s as "realistic"

In [13]:
real_z = []
real_gens = []
for i in range(int(n_samples/2)):
    input, target, condition = model.random_training_set(pairs, input_side, output_side, random_state, DEVICE)
    with torch.no_grad():
        _, _, z, _ = vae(input, target, condition, DEVICE, temperature)
        real_z.append(z)
        real_gens.append(generate(condition, z=z, max_sample=True)[0])

In [14]:
from collections import Counter
c1 = Counter([word for gen in real_gens for word in gen.split()])
c1.most_common(25)

[('UNK', 106),
 ('and', 53),
 ('the', 50),
 ('a', 30),
 ('of', 30),
 ('on', 28),
 ('to', 24),
 ('in', 15),
 ('is', 15),
 ('i', 14),
 ('with', 14),
 ('w', 13),
 ('even', 11),
 ('that', 10),
 ('for', 10),
 ('who', 10),
 ('this', 9),
 ('always', 9),
 ('it', 9),
 ('tour', 8),
 ('into', 8),
 ('be', 8),
 ('but', 8),
 ('you', 7),
 ('new', 7)]

In [15]:
# up until now, `zs` held random zs - now concat with real zs
zs = torch.cat((torch.stack(real_z).squeeze(), torch.stack(zs[:int(n_samples/2)]).squeeze()))

In [16]:
labels = np.ones(n_samples, dtype=int)
labels[range(len(real_z), len(zs))] = -1
embeds = zs

labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

### Realism + Readability
in addition to the realism discriminator, add readability as a conditioning attribute

In [14]:
# https://github.com/shivam5992/textstat/issues/43
from textstat.textstat import textstat

# Score 	 Difficulty
# 90-100 	 Very Easy
# 80-89 	 Easy
# 70-79 	 Fairly Easy
# 60-69 	 Standard
# 50-59 	 Fairly Difficult
# 30-49 	 Difficult
# 0-29 	 Very Confusing

[textstat.flesch_reading_ease(sent) for sent in ["This is a sentence", "To be or not to be", ]]

[92.8, 116.15]

In [26]:
[(' '.join(gen.replace('UNK', '').split()), textstat.text_standard(' '.join(gen.replace('UNK', '').split()))) for gen in np.array(real_gens)[random_state.choice(len(real_gens), 10)]]

[('one of those who dont want to the one of their own who are their own in their own music',
  '4th and 5th grade'),
 ('the remix the production from the original and it sounds like a it with the bass and that sounds like it sounds like it from the chorus',
  '6th and 7th grade'),
 ('we been waiting for the last year while while its soon as waiting for little while we we as we as it',
  '12th and 13th grade'),
 ('one of those who dont want to the one of their own who are their own in their own music',
  '4th and 5th grade'),
 ('we trying to get into an artist is at the one of the and in the song that a part of the is in the trying to into it into a song to artist is just into part into that that or else',
  '14th and 15th grade'),
 ('if youre on the first version of you of know if you hear the version of this is that is to a song',
  '1th and 2th grade'),
 ('we been waiting for the last year while while its soon as waiting for little while we we as we as it',
  '12th and 13th grade'),


### Topical
Prefer certain topics to others

In [9]:
import pytorchtextvae.datasets as ds

def tokenize(line):
    l = line.strip().lstrip().rstrip()
    l = ds.normalize_string(l)
    return l.split(' ')

In [10]:
n_examples = 3

sents = [pair[0] for pair in pairs]
texts = [tokenize(sentence) for sentence in sents]
texts[:n_examples]

[['i',
  'just',
  'can',
  't',
  'explain',
  'that',
  'furthermore',
  'showing',
  'that',
  'he',
  'recognizes',
  'himself',
  'as',
  'a',
  'real',
  'one'],
 ['skream',
  'kept',
  'featured',
  'vocalist',
  'remy',
  'banks',
  'inhale',
  'exhale',
  'lyrics',
  'while',
  'stripping',
  'the',
  'track',
  'of',
  'its',
  'minimalist',
  'garage',
  'like',
  'house',
  'beat',
  'and',
  'replacing',
  'it',
  'with',
  'a',
  'more',
  'propulsive',
  'and',
  'glossier',
  'array',
  'of',
  'synths'],
 ['distinct',
  'pieces',
  'of',
  'instrumentation',
  'build',
  'on',
  'each',
  'other',
  'to',
  'create',
  'a',
  'groove',
  'thats',
  'as',
  'smooth',
  'as',
  'it',
  'is',
  'entrancing']]

In [11]:
from nltk.corpus import stopwords

# remove stop words and words that appear only once
stoplist = [ds.normalize_string(word) for word in stopwords.words('english')]
fillerlist = ['author', 'song_title', 'artist', 'sitename']

texts = [[word for word in text if word not in stoplist and word not in fillerlist] for text in texts]
texts[:n_examples]

[['explain', 'furthermore', 'showing', 'recognizes', 'real', 'one'],
 ['skream',
  'kept',
  'featured',
  'vocalist',
  'remy',
  'banks',
  'inhale',
  'exhale',
  'lyrics',
  'stripping',
  'track',
  'minimalist',
  'garage',
  'like',
  'house',
  'beat',
  'replacing',
  'propulsive',
  'glossier',
  'array',
  'synths'],
 ['distinct',
  'pieces',
  'instrumentation',
  'build',
  'create',
  'groove',
  'thats',
  'smooth',
  'entrancing']]

In [12]:
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(texts)

In [13]:
from gensim.models.ldamodel import LdaModel
import time

start = time.time()
n_topics = 4
passes = 20 # number of passes through documents
iterations = 400 # maximum number of iterations through the corpus when inferring the topic distribution of a corpus.

corpus = [dictionary.doc2bow(text) for text in texts]
# Train the model on the corpus.
lda = LdaModel(corpus, id2word=dictionary, num_topics=n_topics, iterations=iterations, passes=passes)
#lda = LdaModel(corpus, id2word=dictionary, num_topics=n_topics)
print(f'Runtime: {time.time() - start:.2f}s')
lda.print_topics(n_topics)

Runtime: 520.71s


[(0,
  '0.015*"track" + 0.013*"pop" + 0.012*"vocals" + 0.007*"sound" + 0.006*"song" + 0.006*"production" + 0.005*"vocal" + 0.005*"electronic" + 0.005*"like" + 0.005*"guitar"'),
 (1,
  '0.012*"remix" + 0.010*"producer" + 0.010*"new" + 0.008*"single" + 0.007*"singer" + 0.005*"based" + 0.005*"track" + 0.004*"duo" + 0.004*"also" + 0.004*"young"'),
 (2,
  '0.023*"new" + 0.021*"album" + 0.016*"single" + 0.014*"release" + 0.014*"ep" + 0.012*"debut" + 0.011*"track" + 0.011*"released" + 0.010*"year" + 0.010*"first"'),
 (3,
  '0.016*"one" + 0.013*"like" + 0.012*"song" + 0.011*"time" + 0.011*"music" + 0.008*"get" + 0.007*"love" + 0.007*"track" + 0.007*"us" + 0.006*"much"')]

In [14]:
from operator import itemgetter

for i in range(n_examples):
    print(max(lda[corpus[i]],key=itemgetter(1)), ds.normalize_string(sents[i]))

(3, 0.7269966) i just can t explain that furthermore showing that he recognizes himself as a real one
(0, 0.68076444) skream kept featured vocalist remy banks inhale exhale lyrics while stripping the track of its minimalist garage like house beat and replacing it with a more propulsive and glossier array of synths
(0, 0.9242833) distinct pieces of instrumentation build on each other to create a groove thats as smooth as it is entrancing


In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.156851 -0.108565       1        1  28.260656
0     -0.230325  0.104114       2        1  28.121206
2      0.180483 -0.230121       3        1  25.371683
1      0.206693  0.234572       4        1  18.246462, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
238   Default   7020.000000       album   7020.000000  30.0000  30.0000
562   Default   4610.000000     release   4610.000000  29.0000  29.0000
143   Default   4475.000000          ep   4475.000000  28.0000  28.0000
64    Default   7142.000000      single   7142.000000  27.0000  27.0000
208   Default   4528.000000      vocals   4528.000000  26.0000  26.0000
59    Default   3876.000000       debut   3876.000000  25.0000  25.0000
283   Default   4118.000000        time   4118.000000  24.0000  24.0000
278   Default   3481.000000    released   3481.000000  23.0000  23.0000
374   Default   5385.000000         pop   5385.000000  22.0000  22.0000
147   Default  12420.000000         new  12420.000000  21.0000  21.0000
277   Default   2859.000000    producer   2859.000000  20.0000  20.0000
250   Default   4236.000000        year   4236.000000  19.0000  19.0000
144   Default   4953.000000       first   4953.000000  18.0000  18.0000
2     Default   7700.000000         one   7700.000000  17.0000  17.0000
281   Default   2388.000000         via   2388.000000  16.0000  16.0000
16    Default   6925.000000        like   6925.000000  15.0000  15.0000
400   Default   4594.000000       remix   4594.000000  14.0000  14.0000
1092  Default   1722.000000      singer   1722.000000  13.0000  13.0000
486   Default   3290.000000        last   3290.000000  12.0000  12.0000
457   Default   3278.000000         get   3278.000000  11.0000  11.0000
1208  Default   1870.000000        tour   1870.000000  10.0000  10.0000
288   Default   3423.000000       sound   3423.000000   9.0000   9.0000
769   Default   1767.000000     records   1767.000000   8.0000   8.0000
455   Default   1903.000000       vocal   1903.000000   7.0000   7.0000
766   Default   1892.000000          im   1892.000000   6.0000   6.0000
441   Default   2541.000000       based   2541.000000   5.0000   5.0000
800   Default   1749.000000        know   1749.000000   4.0000   4.0000
563   Default   1608.000000  soundcloud   1608.000000   3.0000   3.0000
356   Default   2429.000000  production   2429.000000   2.0000   2.0000
276   Default   2289.000000        good   2289.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
1912   Topic4    338.157227        body    338.946686   1.6989  -6.5567
2318   Topic4    326.044128     putting    326.821594   1.6988  -6.5932
1275   Topic4    311.695038    canadian    312.456604   1.6988  -6.6382
2284   Topic4    319.703369    dropping    320.511261   1.6987  -6.6128
7918   Topic4    295.777191       kanye    296.525574   1.6987  -6.6906
2984   Topic4    303.351562      family    304.126373   1.6986  -6.6653
277    Topic4   2463.327637    producer   2859.773438   1.5520  -4.5709
4750   Topic4    393.125214      french    403.275574   1.6757  -6.4061
400    Topic4   2819.167725       remix   4594.224121   1.2128  -4.4360
2233   Topic4    567.114990          dj    669.873718   1.5347  -6.0396
536    Topic4    770.255554    produced   1004.705566   1.4355  -5.7335
348    Topic4    596.687134       known    744.291016   1.4802  -5.9888
621    Topic4    833.763611   featuring   1216.623535   1.3233  -5.6543
2262   Topic4    499.476807      rapper    600.355225   1.5172  -6.1666
441    Topic4   1254.571045       based   2541.895020   0.9951  -5.2457
1650   Topic4    789.775146          la   1239.842285   1.2502  -5.7085
64     Topic4   1927.198486      single   7142.153320   0.3913  -4.8164
373    Topic4   1000.499390      

In [16]:
from collections import Counter
Counter([max(lda[corpus[i]], key=itemgetter(1))[0] for i in range(len(texts))])

Counter({3: 35568, 0: 28974, 2: 26810, 1: 13115})

In [17]:
out = sorted([f'{max(lda[corpus[i]], key=itemgetter(1))} {ds.normalize_string(sents[i])}\n' for i in range(len(texts))], reverse=True)
with open('pairs_sentence_topics.txt', 'w') as f:
    f.writelines(out)

^using this file to select desirable/preferred topics.

In [18]:
def get_example(i, pairs, input_side, output_side, random_state, device):
    pair = pairs[i]

    inp = model.word_tensor(input_side, pair[0]).to(device)
    target = model.word_tensor(output_side, pair[1]).to(device)
    condition = torch.tensor(pair[2], dtype=torch.float).unsqueeze(0).to(device) if len(pair) == 3 else None

    return inp, target, condition

In [20]:
import numpy as np
import time
from pytorchtextvae import model

good_topics = [0]
bad_topics = [2]

labels = np.zeros(len(texts), dtype=float)
embeds = []
start = time.time()

# debug vars
ts = []
iss = []

for i in range(n_samples):
    pair_i = random_state.choice(len(pairs))
    iss.append(pair_i)
    ts.append(pairs[pair_i][0])
    input, target, condition = get_example(pair_i, pairs, input_side, output_side, random_state, DEVICE)
    with torch.no_grad():
        _, _, z, _ = vae(input, target, condition, DEVICE, temp)
        squashed_condition = vae.decoder.c2h(condition)
        decode_embed = torch.cat([z, squashed_condition], 1)
        embeds.append(decode_embed)
    
    topic_tuple = max(lda[corpus[pair_i]],key=itemgetter(1))
    topic_i = topic_tuple[0]
    topic_p = topic_tuple[1]
    if topic_i in good_topics:
        labels[pair_i] = 1 * topic_p
    elif topic_i in bad_topics:
        labels[pair_i] = -1 * topic_p
    else:
        labels[pair_i] = 0

print(f'runtime: {time.time() - start:.2f}s')
list(zip([labels[j] for j in iss], ts[:n_samples]))

runtime: 130.24s


[(0.0,
  'artist s song_title is a sultry song about someone young is crazy about'),
 (-0.8483790159225464,
  'the record is called the song_title and it will be out on september 18'),
 (0.0,
  'i think it would be pretty sweet to see a vip as clockwork on song_title'),
 (0.0,
  'long story short when jackson tells you that the temperature is rising'),
 (0.0,
  'this track is definitely going to be noticed in the bars and clubs especially when youre in your own zone the song says it all dont bother me'),
 (0.0, 'you can hear the track in the player below from billboard'),
 (0.4947097897529602,
  'coupled with gary gos inspiring summertime lyrics some acoustic strumming and a soaring melody this is sure to please dance fans across the globe'),
 (0.0,
  'the project from the iconic hip hop collective will contain 10 total songs and has performances from the super producer method man raekwon ghostface killa inspektah deck masta killa odb redman sean price and more'),
 (0.6492617130279541,

## Data

In [21]:
batch_size = 16
embed_size = embeds[0].size(1)

In [22]:
from fastai.dataset import *

class LatentDataset(Dataset):
    def __init__(self, embeds, labels): self.embeds,self.labels = embeds,labels
    def __getitem__(self, idx): return A(self.embeds[idx], self.labels[idx])
    def __len__(self): return len(self.embeds)
    
ds = LatentDataset(embeds, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [23]:
md.trn_ds[0], md.trn_ds[0][0].shape

([array([[ 0.36869,  1.79978,  0.86702,  3.8269 ,  1.64594, -0.38677,  0.94261, -0.87616, -1.88315, -1.1856 ,
          -0.4975 , -0.8577 , -2.26257,  0.6806 , -1.96929, -2.61064,  0.67272, -0.67697,  1.23307,  1.71745,
          -0.68316,  0.99863,  0.28309, -1.30979, -1.14636,  0.16227, -0.36618, -3.98761,  0.29842,  2.55866,
          -0.0683 , -1.64817,  2.0768 , -0.39075, -3.25093, -2.79426, -0.04728, -2.3273 , -0.52867,  0.38124,
           1.50383, -2.23158,  0.96316, -1.22391, -1.60539, -0.9431 , -1.04276,  0.28884, -0.66705, -0.43386,
          -3.23542, -0.73141,  0.15726, -0.63864, -1.29561, -0.5971 ,  0.35589,  1.01101,  2.0596 ,  1.01456,
           0.99934,  1.27285, -2.02508, -0.48411,  0.77387, -0.65295,  1.27314,  2.30637,  1.10529, -2.23485,
           1.24869, -1.36013,  1.2846 ,  3.82042,  3.39896,  0.16872, -0.88434, -0.86004,  1.93291, -0.30785,
          -0.69651, -1.53864,  1.44584, -1.65759,  1.28919,  0.64774,  2.63267,  1.99821,  1.06826,  0.24646,
          

## Model

In [24]:
n_hidden = 1024
lr = 3e-4
fixed_genres = torch.FloatTensor(dataset.encode_genres(['edm', 'tropical house'])).to(DEVICE)

In [25]:
import torch.optim as optim
import torch.nn as nn

class LCGAN_D(nn.Module):
    '''Discriminator'''
    def __init__(self, n_embed, n_hidden=n_hidden, n_output=1):
        super(LCGAN_D, self).__init__()
        
        self.i2h = nn.Linear(n_embed, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        self.h2o = nn.Linear(n_hidden, n_output)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, emb):
        x = emb
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        v = self.sigmoid(self.h2o(x))
        
        return v

class LCGAN_G(nn.Module):
    '''Generator'''
    def __init__(self, n_embed, n_hidden=n_hidden):
        super(LCGAN_G, self).__init__()
        self.n_embed = n_embed
        
        self.i2h = nn.Linear(n_embed, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        # hidden-to-gating mechanism
        self.h2g = nn.Linear(n_hidden, 2*n_embed)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, emb):
        x = emb
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        x = self.h2g(x)
        
        # gating mechanism: allow network to remember/forget
        # what it wants to about the original emb(edding) and x
        emb_mid = x[:, self.n_embed:]
        gates = self.sigmoid(x[:, :self.n_embed])
        demb = gates * emb_mid # TODO: why naming?
        emb_prime = (1 - gates)*emb + demb
        
        return emb_prime

In [26]:
class LCGAN(nn.Module):
    def __init__(self, D, G, batch_size=batch_size):
        super(LCGAN, self).__init__()
        self.batch_size = batch_size
        
        self.D = D
        self.G = G

    def train(self):
        self.D.train()
        self.G.train()
        
    def eval(self):
        self.D.eval()
        self.G.eval()
        
    def forward(self, emb=None):       
        if emb is not None:
            # train discriminator
            #embed = to_embed(z, fixed_genres)
            v = self.D(emb)
            return v
        else: # train GAN
            # gaussian random noise
            emb_prior = torch.randn(self.batch_size, self.G.n_embed).to(DEVICE)
            
            emb_prime = self.G(emb_prior)
            v_prime = self.D(emb_prime)
            
            return v_prime

## Training

In [27]:
gan = LCGAN(LCGAN_D(embed_size).to(DEVICE), LCGAN_G(embed_size).to(DEVICE)).to(DEVICE)

In [28]:
import fastai

fastai.core.set_trainable(gan.D, True)
fastai.core.set_trainable(gan.G, True)

opt_d = optim.Adam(gan.D.parameters(), lr=lr)
opt_g = optim.Adam(gan.G.parameters(), lr=lr)

In [29]:
# test what the GAN is doing before any training
for i in range(10):
    print(generate(fixed_genres, gan)[0])

artist returns today with some of the UNK seem to be described as described as most recent eyes
in my UNK is a laid back in as you seems to look listen to the live minutes
UNK UNK UNK UNK has a sort of singing collaboration with right below to catch them wait
to track is a laid back with one of fact that we have to stuck on your space
the UNK UNK is UNK with UNK and as part of the track to keep an full on vocal performance
ever since i grew up in a bit of UNK and i listened to making the way too just last end
with a part of what happens to come back in the right and and rather just some time path
the UNK UNK a UNK that is UNK to UNK back and some of the live show at major series
UNK UNK UNK UNK as a thanks to UNK sultry vocals serves as one of the continues to come tour
artist are the plans for a track called artist and the fact and really made it all together


In [30]:
#from tqdm import tqdm

# adapted from: https://github.com/fastai/fastai/blob/master/courses/dl2/wgan.ipynb
def train(n_iter, alternate=False, first=False):
    gen_iters = 0
    for epoch in trange(n_iter):
        gan.train()
        data_iter = iter(md.trn_dl)
        i, n = 0, len(md.trn_dl)
        
        def train_G():
            ''' Train generator '''
            nonlocal gen_iters
            
            fastai.core.set_trainable(gan.D, False)
            fastai.core.set_trainable(gan.G, True)

            gan.G.zero_grad()

            #print(i, n)
            loss_g = - torch.log(gan()).mean()
            loss_g.backward()
            opt_g.step()
            gen_iters += 1
            
            return loss_g

        def train_D():
            ''' Train discriminator '''
            nonlocal i
            
            fastai.core.set_trainable(gan.D, True)
            fastai.core.set_trainable(gan.G, False)
            d_iters = 100 if (first and (gen_iters < 25) or (gen_iters % 500 == 0)) else 3
            j = 0

            while (j < d_iters) and (i < n):
                j += 1; i += 1
                batch = next(data_iter)
                #print(j, i, batch[0].size(), batch[1].size())
                emb_real = batch[0].to(DEVICE)
                v = gan(emb_real).to(DEVICE)

                gan.D.zero_grad()

                #loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                loss_d = - (batch[1].to(DEVICE) * torch.log(v) + (1.0-batch[1].to(DEVICE)) * torch.log(1.0 - v)).mean()
                loss_d.backward()
                opt_d.step()
                
                pbar.update()
                
            return loss_d
        
        with tqdm(total=n) as pbar:
            while i < n:
                if alternate:
                    # train discriminator
                    loss_d = train_D()
                    # then train generator a little bit
                    loss_g = train_G()
                else:
                    # train generator only
                    i += 1
                    loss_g = train_G()
                    pbar.update()
        
        if alternate:
            print(f'Loss_D {to_np(loss_d)}; Loss_G {to_np(loss_g)}; ')
        else:
            print(f'Loss_G {to_np(loss_g)}; ')

In [31]:
def train_and_generate(gan, n_epoch, genres=fixed_genres, alternate=False, n_sample=10):
    train(n_epoch, alternate)
    res = []
    for i in range(n_sample):
        res.append(generate(genres, gan))
    return res

In [32]:
[res[0] for res in train_and_generate(gan, 1, alternate=True)]

100%|██████████| 13/13 [00:00<00:00, 46.33it/s]
Loss_D 1.1578314115467947e-05; Loss_G 4.556859016418457; 
100%|██████████| 1/1 [00:00<00:00,  3.54it/s]

['aside from the UNK seems to come back to my year and holding on of cause i wild enjoy',
 'with a a track that is coming out of UNK and and UNK seems to have a smile on your face',
 'UNK UNK UNK UNK full of stripped down and followed by the reminiscent of the subject effect',
 'pop trio artist has signed to UNK and and some of the song to thats taken from an solo together',
 'with the release and his debut single called UNK which has one of the three spin below along with four effect',
 'UNK is a part of of songstress back but by artist the UNK reminds me over your radio enjoy',
 'UNK has teamed up with the UNK of UNK and i cant help but in the UNK i tried to give it to an electro pop video',
 'back in the UNK is one of my of eyes and and back to the different things weve cannot enjoy',
 'the fact that that is back with UNK UNK but thats bound to be remixed by artist below',
 'too many other artists artist has teamed up with first song and all over our last UNK minutes']

In [33]:
[res[0] for res in train_and_generate(gan, 1)]

100%|██████████| 13/13 [00:00<00:00, 44.74it/s]
Loss_G 2.212273359298706; 
100%|██████████| 1/1 [00:00<00:00,  3.42it/s]

['artist has nearly UNK months UNK but in the UNK continues to keep an eye on it dj enjoy',
 'artist will self serve as a UNK continues to kick back of the the built over an ethereal enjoy',
 'UNK has teamed up with some of the sounds and a chopped up to and after on its vocal work',
 'artist is meant to come out of of UNK UNK has pick over a deep in let it wild enjoy',
 'UNK UNK is the kind of spin here and its a ability to holding keep in electronic space',
 'UNK UNK has signed to UNK UNK with some of the track to be featured on her vocal verse',
 'back in one of the trio artist has impressed us with a song single hailing from ethereal today',
 'UNK UNK UNK UNK in the UNK has come back to a strong especially when nearly all night apart',
 'from the UNK UNK UNK is bit of to laid back in the a while keep your live atmosphere',
 'its ethereal enjoy']

## Inference with saved models

In [34]:
def save():
    save_state_filename = 'ganG_state.pt'
    torch.save(gan.G.state_dict(), save_state_filename)
    print('Saved as %s' % (save_state_filename))
    
save()

Saved as ganG_state.pt


In [55]:
def gan_generate(vae, condition, n_latent, ganG, max_sample=False, trunc_sample=True):
    with torch.no_grad():
        ganG.eval()
        z = torch.randn(1, n_latent).to(DEVICE)
        decode_embed = to_embed(z, condition).to(DEVICE)
        z_prime = ganG(decode_embed)

        generated = vae.decoder.generate_with_embed(z_prime, 50, temp, DEVICE, max_sample=max_sample, trunc_sample=trunc_sample)
        generated_str = model.float_word_tensor_to_string(output_side, generated)

        EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

        if generated_str.endswith(EOS_str):
            generated_str = generated_str[:-5]

        # flip it back
        return generated_str[::-1], z, z_prime

In [56]:
ganG = LCGAN_G(embed_size).to(DEVICE)
ganG.load_state_dict(torch.load('ganG_state.pt'))

gan_generate(vae, torch.FloatTensor(dataset.encode_genres(['hip hop','pop','pop rap','rap','trap music'])).to(DEVICE), n_latent, ganG)

('a form of UNK and UNK is no wonder throughout a track just interested welcome addition to their music career',
 tensor([[ 0.3426, -0.4434,  0.6537, -1.0167,  0.2749,  0.3956, -1.0323,
           1.9262, -0.2886,  0.0875, -1.1526,  0.5510, -0.6338, -0.5600,
          -1.1005, -0.0089, -0.0115,  0.3233, -1.4833, -0.4620,  0.2851,
          -0.1228, -2.3361, -1.1106,  0.3530,  0.6186, -0.2250, -1.9346,
           0.4420, -1.3540, -0.9850,  1.5157,  1.8120,  0.8877, -1.1195,
           0.1646,  0.2872, -1.1713, -0.0716, -0.2902, -0.3561,  0.2219,
           0.8907, -1.1704, -0.0048, -0.6173,  1.3518,  0.4813, -0.1438,
          -1.4539,  0.2248, -0.2255,  1.8956,  0.4242,  0.5299, -0.6540,
           0.7414, -0.7336, -0.3256,  1.0842,  0.2970, -1.3575, -1.0421,
          -1.7367, -0.6553,  0.8228,  1.5621, -0.5879, -0.6136, -0.1467,
          -0.0047,  1.4218,  1.6615,  0.0985, -0.0368, -0.6737,  0.0706,
          -1.5198, -0.6521, -1.7595,  0.6300,  2.0502,  1.0153, -0.0525,
           

## Extras

retrain discriminator with new samples... `z_prime`s that the discriminator is still not rejecting strongly enough

In [36]:
labels = np.ones(len(g2), dtype=int)
gens_lose = list(set([i for b in banned for i in np.where([b in res[0].split() for res in g2])[0]]))
#gens_keep = list(set(range(len(g2))) - gens_lose)
labels[gens_lose] = -1
zs_keep = np.array([res[2] for res in g2], dtype=object)

labels        

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [32]:
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [33]:
[res[0] for res in train_and_generate(gan, 1, alternate=True)]

100%|██████████| 7/7 [00:00<00:00, 284.11it/s]
Loss_D 1.193210244178772; Loss_G 1.1896806955337524; 


['artist has teams up with a a one of the UNK and it would be taken from i don no feat',
 'this with a remix of UNK and trying to take of the UNK theres no feat',
 'the remix of UNK has been trying to take of the UNK theres no feat',
 'UNK is a listen to the UNK and coming side of what to look at feat',
 'a new track called UNK and serves as one of the trying would be ready for at times',
 'artist has up with a UNK and one of the track will be inspired as no feat',
 'artist is back in the likes of UNK and serves as a no times',
 'artist who reminds back to a more stuck on the UNK and theres many feat',
 'he continues to serves as one of the week what i look no feat',
 'with a layers of UNK and and UNK of the perfect trying i look at feat']